# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [2]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [3]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [4]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [5]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh8fkwpAAEIAAA9Y2pcAAAAI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":931112759,"wgRevisionId":931112759,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Article

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [6]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [7]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [8]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [9]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF[:10])

                                      paragraph-text
0  Content analysis is a research method for stud...
1  Practices and philosophies of content analysis...
2  Computers are increasingly used in content ana...
3  Content analysis is best understood as a broad...
4  The simplest and most objective form of conten...
5  A further step in analysis is the distinction ...
6  More generally, content analysis is research u...
7  By having contents of communication available ...
8  Robert Weber notes: "To make valid inferences ...
9  There are five types of texts in content analy...


Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [10]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [11]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [12]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [13]:
#The subject for my MA thesis is Asian culture.
#Extracting content data from 'https://en.wikipedia.org/wiki/Culture_of_Asia' 
wikipedia_Culture_of_Asia = 'https://en.wikipedia.org/wiki/Culture_of_Asia'
wikiContentRequest = requests.get(wikipedia_Culture_of_Asia)
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')

#Extracting paragraphs
ccontentPTags = wikiContentSoup.body.findAll('p')
for pTag in ccontentPTags[:3]:
    print(pTag.text)
#Making dataframe with the paragraphs
ccontentParagraphs = []
for pTag in ccontentPTags:
    ccontentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#Now I will split the paragraphs into cleaned sentences.
#coding from: https://stackoverflow.com/questions/25735644/python-regex-for-splitting-text-into-sentences-sentence-tokenizing/25735848
mylist = []
for x in range(50):
    s = ccontentParagraphs[x]
    m = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', s)
    mylist.append(m) 

#Now I have a list of lists that consist of sentences from each paragraph.
#To convert this list to a dataframe, I need to make the list of lists into one list.
#Flattening out the list, coding from https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flat_list = []
for sublist in mylist:
    for item in sublist:
        flat_list.append(item)

#Converting the list into a dataframe
df_sentences = pandas.DataFrame(flat_list)
print(df_sentences)



The culture of Asia encompasses the collective and diverse customs and traditions of art, architecture, music, literature, lifestyle, philosophy, politics and religion that have been practiced and maintained by the numerous ethnic groups of the continent of Asia since prehistory. Identification of a specific culture of Asia or universal elements among the colossal diversity that has emanated from multiple cultural spheres and three of the four ancient River valley civilizations is complicated. However, the continent is commonly divided into six geographic sub-regions, that are characterized by perceivable commonalities, like religion, language and relative ethnic homogeneity. These regions are Central Asia, East Asia, North Asia, South Asia, Southeast Asia and West Asia.[1]

As the largest, most populous continent and rich in resources, Asia is home to several of the world's oldest civilizations, that produced the majority of the great religious systems, the oldest known recorded myt


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [14]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [15]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,6
7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,8
9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [16]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [17]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,None,None


# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>

In [18]:
#Extracting content from 'https://en.wikipedia.org/wiki/Culture_of_Asia'
wikipedia_Culture_of_Asia = 'https://en.wikipedia.org/wiki/Culture_of_Asia'
wikiCultureRequest = requests.get(wikipedia_Culture_of_Asia)
wikiCultureSoup = bs4.BeautifulSoup(wikiCultureRequest.text, 'html.parser')

#Organizing the content by paragraphs 
ccontentPTags = wikiCultureSoup.body.findAll('p')
for pTag in ccontentPTags[:3]:
    print(pTag.text)
#Making a pandas dataframe with the paragraphs
ccontentParagraphs = []
for pTag in ccontentPTags:
    ccontentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))
#Converting to a DataFrame
ccontentParagraphsDF = pandas.DataFrame({'paragraph-text' : ccontentParagraphs})
print(ccontentParagraphsDF[:10])





The culture of Asia encompasses the collective and diverse customs and traditions of art, architecture, music, literature, lifestyle, philosophy, politics and religion that have been practiced and maintained by the numerous ethnic groups of the continent of Asia since prehistory. Identification of a specific culture of Asia or universal elements among the colossal diversity that has emanated from multiple cultural spheres and three of the four ancient River valley civilizations is complicated. However, the continent is commonly divided into six geographic sub-regions, that are characterized by perceivable commonalities, like religion, language and relative ethnic homogeneity. These regions are Central Asia, East Asia, North Asia, South Asia, Southeast Asia and West Asia.[1]

As the largest, most populous continent and rich in resources, Asia is home to several of the world's oldest civilizations, that produced the majority of the great religious systems, the oldest known recorded myt

In [19]:
def recursion_function(i):
    cotherPAgeURLS = []
    #We also want to know where the links come from so we also will get:
    #the paragraph number
    #the word the link is in
    for paragraphNum, pTag in enumerate(ccontentPTags):
        #we only want hrefs that link to wiki pages
        tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        for aTag in tagLinks:
            #We need to extract the url from the <a> tag
            relurl = aTag.get('href')
            linkText = aTag.text
            #wikipedia_base_url is the base we can use the urllib joining function to merge them
            #Giving a nice structured tupe like this means we can use tuple expansion later
            cotherPAgeURLS.append((
                urllib.parse.urljoin(wikipedia_base_url, relurl),
                paragraphNum,
                linkText,
            ))
    return(cotherPAgeURLS)
    if i< 100:
        recursion_function(i+1)
            
re_cotherPAgeURLS=recursion_function(0)
print(re_cotherPAgeURLS)

[('https://en.wikipedia.org/wiki/Ethnic_group', 1, 'ethnic groups'), ('https://en.wikipedia.org/wiki/Asia', 1, 'Asia'), ('https://en.wikipedia.org/wiki/Prehistoric_Asia', 1, 'prehistory'), ('https://en.wikipedia.org/wiki/River_valley_civilization', 1, 'four ancient River valley civilizations'), ('https://en.wikipedia.org/wiki/Central_Asia', 1, 'Central Asia'), ('https://en.wikipedia.org/wiki/East_Asia', 1, 'East Asia'), ('https://en.wikipedia.org/wiki/North_Asia', 1, 'North Asia'), ('https://en.wikipedia.org/wiki/South_Asia', 1, 'South Asia'), ('https://en.wikipedia.org/wiki/Southeast_Asia', 1, 'Southeast Asia'), ('https://en.wikipedia.org/wiki/Myth', 2, 'myths'), ('https://en.wikipedia.org/wiki/Codex', 2, 'codices'), ('https://en.wikipedia.org/wiki/Pan-Asianism', 3, 'Pan-Asian'), ('https://en.wikipedia.org/wiki/Ian_Morris_(historian)', 5, 'Ian Morrison'), ('https://en.wikipedia.org/wiki/Middle_East', 5, 'Middle East'), ('https://en.wikipedia.org/wiki/Fertile_Crescent', 5, 'Fertile Cre

In [102]:
ccontentParagraphsDF['source'] = [wikipedia_Culture_of_Asia] * len(ccontentParagraphsDF['paragraph-text'])
ccontentParagraphsDF['paragraph-number'] = range(len(ccontentParagraphsDF['paragraph-text']))

ccontentParagraphsDF

,paragraph-text,source,paragraph-number
0,\n,https://en.wikipedia.org/wiki/Culture_of_Asia,0
1,The culture of Asia encompasses the collective...,https://en.wikipedia.org/wiki/Culture_of_Asia,1
2,"As the largest, most populous continent and ri...",https://en.wikipedia.org/wiki/Culture_of_Asia,2
3,"However, Asia's enormous size separates the va...",https://en.wikipedia.org/wiki/Culture_of_Asia,3
4,Asia's various modern cultural and religious s...,https://en.wikipedia.org/wiki/Culture_of_Asia,4
5,West Asia (or Southwest Asia as Ian Morrison p...,https://en.wikipedia.org/wiki/Culture_of_Asia,5
6,"South Asia, India and the Indosphere emanate f...",https://en.wikipedia.org/wiki/Culture_of_Asia,6
7,The East Asian cultural sphere developed from ...,https://en.wikipedia.org/wiki/Culture_of_Asia,7
8,Central Asia has also absorbed influences from...,https://en.wikipedia.org/wiki/Culture_of_Asia,8
9,The cultural spheres are not mutually disjoint...,https://en.wikipedia.org/wiki/Culture_of_Asia,9


In [103]:
ccontentParagraphsDF['source-paragraph-number'] = [None] * len(ccontentParagraphsDF['paragraph-text'])
ccontentParagraphsDF['source-paragraph-text'] = [None] * len(ccontentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

In [104]:
for urlTuple in re_cotherPAgeURLS[:3]:
    ccontentParagraphsDF = ccontentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
ccontentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,\n,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
1,1,The culture of Asia encompasses the collective...,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
2,2,"As the largest, most populous continent and ri...",https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
3,3,"However, Asia's enormous size separates the va...",https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
4,4,Asia's various modern cultural and religious s...,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
5,5,West Asia (or Southwest Asia as Ian Morrison p...,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
6,6,"South Asia, India and the Indosphere emanate f...",https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
7,7,The East Asian cultural sphere developed from ...,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
8,8,Central Asia has also absorbed influences from...,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None
9,9,The cultural spheres are not mutually disjoint...,https://en.wikipedia.org/wiki/Culture_of_Asia,None,None


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [ ]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [ ]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [ ]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [9]:
example_text_file = 'sometextfile.txt'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [10]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Let's start by looking at the Shakespeare files in the `data` directory 

In [ ]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [13]:
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@R�Q`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [14]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [15]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [17]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [ ]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [29]:
#Extracting textual content from the PDF that was previously used.
PDFcontent = readPDF(infoExtractionBytes)

#Extracting sentences 
s = PDFcontent
m = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', s)

#making a pandas dataframe of sentences from the first paragraph
PDFcontentDF_sentence = pandas.DataFrame({'sentence-text' : m})
print(PDFcontentDF_sentence[:20])

                                        sentence-text
0                     Speech and Language Processing.
1                          Daniel Jurafsky & James H.
2   Martin.\nrights reserved.\n\nDraft of November...
3   His-\ntorical data about stock prices is easy ...
4   You will need to know at least the name of the...
5   Fortunately, these can be all found in news ar...
6   American Airlines, a unit of AMR Corp., immedi...
7   United, a unit of UAL\nCorp., said the increas...
8   This process of information extraction (IE), t...
9   The task of named entity recognition (NER) is ...
10  What constitutes a named\nentity type is appli...
11  We’ll defer discussion of this task of\ncorefe...
12  Relation extraction has close links to\npopula...
13  We’ll also need to perform event coreference t...
14  The problem of temporal expression normalizati...
15  In our sample task, this will allow us to link...
16  The task of tem-\nplate ﬁlling is to ﬁnd such ...
17  These slot-ﬁllers may co

### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

